In [1]:
import logging
import datetime

import numpy as np
import torch
from torch.nn.utils import parameters_to_vector
import torch.optim as optim
from torchinfo import summary

import matplotlib.pyplot as plt

import config
import modules.dataloaders as dataloaders

#import modules.models_bed_evolution.bed_04_brevitas as cnv_model
# import modules.models_bed_evolution.bed_04_brevitas_mul4 as cnv_model

# import modules.models_bed_evolution.bed_05_brevitas_fpga as cnv_model
# import modules.models_bed_evolution.bed_05_brevitas_fpga_old as cnv_model

import modules.models_bed_evolution.bed_05_brevitas_fpga_old_small_big as cnv_model


import modules.loss as loss
import modules.metrics as metrics
import modules.train_epoch as train_epoch
import modules.val_epoch as val_epoch
import modules.utils as utils

/opt/conda/envs/pytorch_brevitas/lib/python3.10/site-packages/onnxscript/converter.py:820: FutureWarning: 'onnxscript.values.Op.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
/opt/conda/envs/pytorch_brevitas/lib/python3.10/site-packages/onnxscript/converter.py:820: FutureWarning: 'onnxscript.values.OnnxFunction.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()


# Logger

In [2]:
log_path = config.LOGS_FOLDER

logger = logging.getLogger("GonLogger")
logger.propagate = False
logger.setLevel(logging.INFO)
file_handler = logging.FileHandler(log_path + 'logfile.log')
formatter = logging.Formatter('%(message)s')
file_handler.setFormatter(formatter)

# add file handler to logger
logger.addHandler(file_handler)

logger.info('BED Classifier.\n' +  
            '\tOne Head.\n' +
            '\tWeighted for Precision.\n' +
            '\tBrevitas Default.\n'+ 
            '\tDataset images divided by 255.\n')

# Hyperparameters Log

In [3]:
''' ============================
    Print Config Values
============================ '''
print(f'Training Brevitas Model = {config.BREVITAS_MODEL}')
print('\nDatasets Length')
print(f'\tTrain and Val: {"Full" if config.DS_LEN == None else config.DS_LEN}')
print(f'\nLoad Model: {config.LOAD_MODEL}')
if (config.LOAD_MODEL == True):
    print(f'\tModel: {config.LOAD_MODEL_FILE}')
print(f'Device: {config.DEVICE}')
print('Optimizer:')
print(f'\tLearning Rate: {config.LEARNING_RATE}')
print(f'\tWeight Decay: {config.WEIGHT_DECAY}')
print('Scheduler:')
print(f'\tScheduler factor: {config.FACTOR}')
print(f'\tScheduler patience: {config.PATIENCE}')
print(f'\tScheduler threshold: {config.THRES}')
print(f'\tScheduler min learning rate: {config.MIN_LR}')
print(f'Batch Size: {config.BATCH_SIZE}')
print(f'Num Workers: {config.NUM_WORKERS}')
print(f'Pin Memory: {config.PIN_MEMORY}')
print(f'Epochs: {config.EPOCHS}')
print('\nIMG DIMS:')
print(f'\tWidth: {config.IMG_W}\n\tHeight: {config.IMG_H}')
print('\nBrevitas Config:')
print(f'\tFixed Point: {config.FIXED_POINT}')
print(f'\tWeights Bit Width: {config.WEIGHTS_BIT_WIDTH}')
print(f'\tBig Layers Weights Bit Width: {config.BIG_LAYERS_WEIGHTS_BIT_WIDTH}')
print(f'\tBias Bit Width: {config.BIAS_BIT_WIDTH}')
print(f'\tActivations Bit Width: {config.ACTIVATIONS_BIT_WIDTH}')

logger.info(f'Training Brevitas Model = {config.BREVITAS_MODEL}')
logger.info('\nDatasets Length')
logger.info(f'\tTrain and Val: {"Full" if config.DS_LEN == None else config.DS_LEN}')
logger.info(f'\nLoad Model: {config.LOAD_MODEL}')
if (config.LOAD_MODEL == True):
    logger.info(f'\tModel: {config.LOAD_MODEL_FILE}')
logger.info(f'\nDevice: {config.DEVICE}')
logger.info('Optimizer:')
logger.info(f'\tLearning Rate: {config.LEARNING_RATE}')
logger.info(f'\tWeight Decay: {config.WEIGHT_DECAY}')
logger.info('Scheduler:')
logger.info(f'\tScheduler factor: {config.FACTOR}')
logger.info(f'\tScheduler patience: {config.PATIENCE}')
logger.info(f'\tScheduler threshold: {config.THRES}')
logger.info(f'\tScheduler min learning rate: {config.MIN_LR}')
logger.info(f'\nBatch Size: {config.BATCH_SIZE}')
logger.info(f'Num Workers: {config.NUM_WORKERS}')
logger.info(f'Pin Memory: {config.PIN_MEMORY}')
logger.info(f'Epochs: {config.EPOCHS}')
logger.info('\nIMG DIMS:')
logger.info(f'\tWidth: {config.IMG_W}\n\tHeight: {config.IMG_H}')
logger.info('\nBrevitas Config:')
logger.info(f'\tFixed Point: {config.FIXED_POINT}')
logger.info(f'\tWeights Bit Width: {config.WEIGHTS_BIT_WIDTH}')
logger.info(f'\tBig Layers Weights Bit Width: {config.BIG_LAYERS_WEIGHTS_BIT_WIDTH}')
logger.info(f'\tBias Bit Width: {config.BIAS_BIT_WIDTH}')
logger.info(f'\tActivations Bit Width: {config.ACTIVATIONS_BIT_WIDTH}')

Training Brevitas Model = True

Datasets Length
	Train and Val: 128

Load Model: False
Device: cuda
Optimizer:
	Learning Rate: 0.001
	Weight Decay: 0.001
Scheduler:
	Scheduler factor: 0.8
	Scheduler patience: 3
	Scheduler threshold: 0.001
	Scheduler min learning rate: 1e-06
Batch Size: 64
Num Workers: 8
Pin Memory: True
Epochs: 3

IMG DIMS:
	Width: 230
	Height: 230

Brevitas Config:
	Fixed Point: True
	Weights Bit Width: 4
	Big Layers Weights Bit Width: 2
	Bias Bit Width: 4
	Activations Bit Width: 4


# Dataloaders

In [4]:
train_loader = dataloaders.get_train_loader()
val_loader = dataloaders.get_val_loader()


TRAIN DFIRE dataset
DFire Removed wrong images: 0
DFire empty images: 56
DFire only smoke images: 37
DFire only fire images: 8
DFire smoke and fire images: 27

Train DFire dataset len: 128

TRAIN FASDD UAV dataset
FASDD Removed wrong images: 0
FASDD empty images: 58
FASDD only smoke images: 22
FASDD only fire images: 2
FASDD smoke and fire images: 46

Train FASDD UAV dataset len: 128

VAL FASDD UAV dataset
FASDD Removed wrong images: 0
FASDD empty images: 59
FASDD only smoke images: 26
FASDD only fire images: 2
FASDD smoke and fire images: 41

Val FASDD UAV dataset len: 128

TRAIN FASDD CV dataset
FASDD Removed wrong images: 0
FASDD empty images: 53
FASDD only smoke images: 36
FASDD only fire images: 19
FASDD smoke and fire images: 20

Train FASDD CV dataset len: 128

Val FASDD CV dataset
FASDD Removed wrong images: 0
FASDD empty images: 50
FASDD only smoke images: 32
FASDD only fire images: 20
FASDD smoke and fire images: 26

Val FASDD CV dataset len: 128

Concatenate Train DFire and

### Plot Some Train Pictures

In [5]:
for i, (img, label) in enumerate(train_loader):

    plt.subplots(8,4, figsize=(8, 16))
    
    for idx in range(config.BATCH_SIZE):
        plt.subplot(8, 4, idx+1)
        plt.imshow(img[idx].permute(1, 2, 0))
        title = ""
        if label[idx][0] == 1 and label[idx][1] == 1:
            title += "Smoke and Fire"
        elif label[idx][0] == 1 and label[idx][1] == 0:
            title += "Only Smoke"
        elif label[idx][0] == 0 and label[idx][1] == 1:
            title += "Only Fire"
        else:
            title += "Empty"
        plt.title(title)
        
        if (idx == 31):
            break
    plt.tight_layout()
    plt.savefig(config.RUN_FOLDER + 'train_pictures.png')
    #plt.show()
    plt.close()
    break

### Plot Some Val Pictures

In [6]:
for i, (img, label) in enumerate(val_loader):

    plt.subplots(8,4, figsize=(8, 16))
    
    for idx in range(config.BATCH_SIZE):
        plt.subplot(8, 4, idx+1)
        plt.imshow(img[idx].permute(1, 2, 0))
        title = ""
        if label[idx][0] == 1 and label[idx][1] == 1:
            title += "Smoke and Fire"
        elif label[idx][0] == 1 and label[idx][1] == 0:
            title += "Only Smoke"
        elif label[idx][0] == 0 and label[idx][1] == 1:
            title += "Only Fire"
        else:
            title += "Empty"
        plt.title(title)
        
        if (idx == 31):
            break
    plt.tight_layout()
    plt.savefig(config.RUN_FOLDER + 'val_pictures.png')
    plt.close()
    break

# Load Model

In [7]:
#model = cnv_model.BED_AIMET_FPGA().to(config.DEVICE)

# model = cnv_model.BED_AIMET_FPGA_MUL4().to(config.DEVICE)

# model = cnv_model.BED_AIMET_FPGA_MANUAL().to(config.DEVICE)

# model = cnv_model.BED_AIMET_FPGA_MANUAL_OLD().to(config.DEVICE)

model = cnv_model.BED_AIMET_FPGA_MANUAL_OLD_SMALL_BIG().to(config.DEVICE)

### Optimizer and Scheduler

In [8]:
optimizer = optim.Adam(model.parameters(), 
                       lr=config.LEARNING_RATE, 
                       weight_decay=config.WEIGHT_DECAY)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 
                                                 mode='min',
                                                 factor=config.FACTOR, 
                                                 patience=config.PATIENCE, 
                                                 threshold=config.THRES, 
                                                 threshold_mode='abs',
                                                 min_lr=config.MIN_LR)



### Parameters

In [9]:
n_trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'\nTrainable parameters = {n_trainable}')
logger.info(f'\nTrainable parameters = {n_trainable}')

n_params = parameters_to_vector(model.parameters()).numel()
print(f'Total parameters = {n_params}\n')
logger.info(f'Total parameters = {n_params}\n')


Trainable parameters = 64353
Total parameters = 64353



### Check Model Shape: Random Input

In [10]:
dummy_input = np.random.rand(4, config.NUM_CHANNELS, config.IMG_H, config.IMG_W)
dummy_input = torch.tensor(dummy_input, dtype=torch.float32, device=config.DEVICE)
out_test = model(dummy_input)
print(f'Model shape is {out_test}')
#print(f'BED Model Arquitecture\n{cnv_model}')

/opt/conda/envs/pytorch_brevitas/lib/python3.10/site-packages/torch/_tensor.py:1362: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at /opt/conda/conda-bld/pytorch_1702400410390/work/c10/core/TensorImpl.h:1900.)
  return super().rename(names)


Model shape is tensor([[ 1.2696,  0.6236],
        [-0.3842,  0.3328],
        [ 0.4416,  0.2719],
        [-2.3246, -0.2307]], device='cuda:0', grad_fn=<AddBackward0>)


### Torchinfo

In [11]:
print(summary(model, input_size=(1, config.NUM_CHANNELS, config.IMG_H, config.IMG_W)))
logger.info(summary(model, input_size=(1, config.NUM_CHANNELS, config.IMG_H, config.IMG_W)))

Layer (type:depth-idx)                                                      Output Shape              Param #
BED_AIMET_FPGA_MANUAL_OLD_SMALL_BIG                                         [1, 2]                    --
├─Sequential: 1-1                                                           [1, 64, 20, 20]           62,173
│    └─QuantIdentity: 2-1                                                   [1, 3, 230, 230]          --
│    │    └─ActQuantProxyFromInjector: 3-1                                  [1, 3, 230, 230]          --
│    │    └─ActQuantProxyFromInjector: 3-2                                  [1, 3, 230, 230]          1
├─Sequential: 1-36                                                          --                        (recursive)
│    └─QuantReLU: 2-51                                                      --                        (recursive)
│    │    └─ActQuantProxyFromInjector: 3-88                                 --                        (recursive)
├─Sequential: 1-37  

# Loss Function

In [12]:
if config.LOSS_FN == "BCE":
    print(f'Loss Function: BCE')
    logger.info(f'\nLoss Function: BCE')
    print(f'Smoke Precision Weight: {config.SMOKE_PRECISION_WEIGHT}')
    logger.info(f'Smoke Precision Weight: {config.SMOKE_PRECISION_WEIGHT}')
    loss_fn = loss.BCE_LOSS(device=config.DEVICE, smoke_precision_weight=config.SMOKE_PRECISION_WEIGHT)
else:
    print("Wrong loss function")
    logger.info("Wrong loss function")
    raise SystemExit("Wrong loss function")

Loss Function: BCE
Smoke Precision Weight: 0.8


# Loggers and Plotters for Losses and Metrics

In [13]:
train_losses_logger = utils.LogLosses()
train_metrics_logger = utils.LogMetrics()
lr_logger = utils.LogLR(log_path=config.PLOTS_FOLDER)

val_losses_logger = utils.LogLosses()
val_metrics_logger = utils.LogMetrics()

loss_plotter = utils.PlotMetrics(log_path=config.PLOTS_FOLDER, model_name=config.MODEL, loss_or_metric='Loss')
metrics_plotter = utils.PlotMetrics(log_path=config.PLOTS_FOLDER, model_name=config.MODEL, loss_or_metric='Metric')

# Main Function to Train

In [14]:
def train_loop(model, start_epoch=0, epochs_to_train=config.EPOCHS):

    ''' ==============================================================
                                TRAINING LOOP
    ============================================================== '''
    start = datetime.datetime.now()
    start_time = start.strftime("%H:%M:%S")
    print(f'\n***Start Training: {start_time}\n')
    logger.info(f'\n***Start Training: {start_time}\n')
    
    # Start with infinite validation loss
    best_valid_loss = np.inf
    best_smoke_precision = 0. #torch.tensor([0.])
    smoke_f1_min_save = 0.9 #torch.tensor([0.9])
    best_mean_f1 = 0.

    if start_epoch == 0:
        epochs_plot = []
    else:
        epochs_plot = [i for i in range(start_epoch)]    

    end_epoch = start_epoch + epochs_to_train
        
    for epoch in range(start_epoch, end_epoch):

        print(f'\n=== EPOCH {epoch}/{end_epoch-1} ===')
        logger.info(f'\n=== EPOCH {epoch}/{end_epoch-1} ===')
        
        #====================== TRAINING ========================#
        current_lr = train_epoch.get_lr(optimizer=optimizer)
        logger.info(f'Learning Rate = {current_lr}\n')
        lr_logger.log_lr(current_lr)
                
        train_losses, train_metrics = train_epoch.train_fn(
            loader=train_loader, 
            model=model, 
            optimizer=optimizer, 
            loss_fn=loss_fn,
            device=config.DEVICE)
        
        train_losses_logger.update_metrics(train_losses)
        train_metrics_logger.update_metrics(train_metrics)
                
        logger.info(utils.print_metrics_to_logger("TRAIN Stats", train_losses, train_metrics))
        
        #===================== VALIDATING =======================#
        with torch.no_grad():
            val_losses, val_metrics = val_epoch.eval_fn(
                loader=val_loader, 
                model=model,                         
                loss_fn=loss_fn,
                device=config.DEVICE)
            
            scheduler.step(val_losses['Total'])
            
            val_losses_logger.update_metrics(val_losses)
            val_metrics_logger.update_metrics(val_metrics)

            logger.info(utils.print_metrics_to_logger("VAL Stats", val_losses, val_metrics))
            
        epochs_plot.append(epoch)

        loss_plotter.plot_all_metrics(
            train_losses_logger.get_metrics(),
            val_losses_logger.get_metrics(),
            epochs_plot)

        metrics_plotter.plot_all_metrics(
            train_metrics_logger.get_metrics(),
            val_metrics_logger.get_metrics(),
            epochs_plot)

        lr_logger.plot_lr(epochs_plot)
        #======================= SAVING =========================#
        if ( (epoch+1) % 5 ) == 0:
            save_name = config.WEIGHTS_FOLDER + config.MODEL + '_classifier__5epoch.pt'
            utils.save_checkpoint(epoch, model, optimizer, scheduler, save_name) 
            
        if best_valid_loss > val_losses['Total']:
            best_valid_loss = val_losses['Total']
            print(f"\nSaving model with new best validation loss: {best_valid_loss:.4f}")
            logger.info(f"Saving model with new best validation loss: {best_valid_loss:.4f}")
            save_name = config.WEIGHTS_FOLDER + config.MODEL + '_classifier__' + 'best_loss'  + '.pt'
            utils.save_checkpoint(epoch, model, optimizer, scheduler, save_name) 
            if config.BREVITAS_MODEL == True:
                save_onnx = config.ONNX_FOLDER + config.MODEL + '_classifier__' + 'best_loss'  #+ '.onnx'
                utils.export_onnx(model, (1, config.NUM_CHANNELS, config.IMG_H, config.IMG_W), save_onnx, config.DEVICE)

        # # Save model if precision increases and F1 > 0.9
        # if ( best_smoke_precision < val_metrics['Precision'][0] ) and ( val_metrics['F1'][0] > smoke_f1_min_save ) :
        #     best_smoke_precision = val_metrics['Precision'][0]
        #     print(f"\nSaving model with new best smoke precision: {best_smoke_precision:.4f}")
        #     logger.info(f"Saving model with new best smoke precision: {best_smoke_precision:.4f}")
        #     save_precision_name = f'best_smoke__precision={np.round(best_smoke_precision, decimals=4)}__epoch={epoch}'
        #     save_name = config.WEIGHTS_FOLDER + config.MODEL + '_classifier__' + save_precision_name + '.pt'
        #     utils.save_checkpoint(epoch, model, optimizer, scheduler, save_name)  
        #     if config.BREVITAS_MODEL == True:
        #         save_onnx = config.ONNX_FOLDER + config.MODEL + '_classifier__' + save_precision_name #+ '.onnx'
        #         utils.export_onnx(model, (1, config.NUM_CHANNELS, config.IMG_H, config.IMG_W), save_onnx, config.DEVICE)
            
        # Save model if precision > 0.9 and recall > 0.9
        # if ( val_metrics['Precision'][0] > 0.9 ) and ( val_metrics['Recall'][0] > 0.9 ) :
        #     print("\nSaving model with precision > 0.9 and recall > 0.9")
        #     logger.info("Saving model with precision > 0.9 and recall > 0.9")
        #     save_pre_name = f'smoke__precision={np.round(val_metrics["Precision"][0], decimals=4)}__' 
        #     save_rec_name = f'recall={np.round(val_metrics["Recall"][0], decimals=4)}__'
        #     save_pre_rec_name = save_pre_name + save_rec_name + f'epoch={epoch}'
        #     save_name = config.WEIGHTS_FOLDER + config.MODEL + '_classifier__' + save_pre_rec_name + '.pt'
        #     utils.save_checkpoint(epoch, model, optimizer, scheduler, save_name) 
        #     if config.BREVITAS_MODEL == True:
        #         save_onnx = config.ONNX_FOLDER + config.MODEL + '_classifier__' + save_pre_rec_name #+ '.onnx'
        #         utils.export_onnx(model, (1, config.NUM_CHANNELS, config.IMG_H, config.IMG_W), save_onnx, config.DEVICE)

        # Save model if best mean F1 increases
        val_f1_mean = (val_metrics['F1'][0] + val_metrics['F1'][1]) / 2
        if (val_f1_mean > best_mean_f1) :
            best_mean_f1 = val_f1_mean
            print(f'Saving model with best Mean F1: {best_mean_f1:.4f}')
            logger.info(f'Saving model with best Mean F1: {best_mean_f1:.4f}')
            save_f1_name = 'best_mean_F1'
            save_name = config.WEIGHTS_FOLDER + config.MODEL + '_classifier__' + save_f1_name + '.pt'
            utils.save_checkpoint(epoch, model, optimizer, scheduler, save_name) 
            if config.BREVITAS_MODEL == True:
                save_onnx = config.ONNX_FOLDER + config.MODEL + '_classifier__' + save_f1_name #+ '.onnx'
                utils.export_onnx(model, (1, config.NUM_CHANNELS, config.IMG_H, config.IMG_W), save_onnx, config.DEVICE)
        
    logger.info('Saving last model')   
    torch.save(model.state_dict(), config.WEIGHTS_FOLDER + 'last_' + config.MODEL + '_classifier.pt') 
    
    #======================= FINISH =========================#
    end = datetime.datetime.now()
    end_time = end.strftime("%H:%M:%S")
    print(f'\n***Script finished: {end_time}\n')  
    print(f'Time elapsed: {end-start}')
    logger.info(f'\n***Script finished: {end_time}\n')  
    logger.info(f'Time elapsed: {end-start}')
    
    return model

In [15]:
# print(len(val_losses_logger.total))

# Training Loop

In [16]:
if __name__ == "__main__":
    
    print("Starting script\n")
    logger.info("Starting script\n")
    
    model = train_loop(model)

Starting script


***Start Training: 19:39:12


=== EPOCH 0/2 ===
Learning Rate = 0.001



Training: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:08<00:00,  1.22it/s]


Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
84.638      |40.739      |43.899      



Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  3.17it/s]


Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
81.673      |39.596      |42.077      
SMOKE -> Precision: 0.0000 - Recall: 0.0000 - Accuracy: 0.5104 - F1: 0.0000
FIRE -> Precision: 0.0000 - Recall: 0.0000 - Accuracy: 0.6979 - F1: 0.0000

Saving model with new best validation loss: 81.6730
Model exported to ONNX: experiments_bed_evolution/70_brevitas__230_manual_old_SmallBig__128_ds/onnx/BED_FPGA_230_classifier__best_loss

=== EPOCH 1/2 ===
Learning Rate = 0.001



...

Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
80.654      |40.001      |40.654      
SMOKE -> Precision: 0.0000 - Recall: 0.0000 - Accuracy: 0.5104 - F1: 0.0000
FIRE -> Precision: 0.0000 - Recall: 0.0000 - Accuracy: 0.6979 - F1: 0.0000

Saving model with new best validation loss: 80.6543
Model exported to ONNX: experiments_bed_evolution/70_brevitas__230_manual_old_SmallBig__128_ds/onnx/BED_FPGA_230_classifier__best_loss

=== EPOCH 2/2 ===
Learning Rate = 0.001



Training: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:08<00:00,  1.23it/s]


Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
68.867      |37.784      |31.083      



Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  3.14it/s]


Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
80.244      |41.026      |39.218      
SMOKE -> Precision: 0.4896 - Recall: 1.0000 - Accuracy: 0.4896 - F1: 0.6573
FIRE -> Precision: 0.0000 - Recall: 0.0000 - Accuracy: 0.6979 - F1: 0.0000

Saving model with new best validation loss: 80.2439
Model exported to ONNX: experiments_bed_evolution/70_brevitas__230_manual_old_SmallBig__128_ds/onnx/BED_FPGA_230_classifier__best_loss
Saving model with best Mean F1: 0.3287
Model exported to ONNX: experiments_bed_evolution/70_brevitas__230_manual_old_SmallBig__128_ds/onnx/BED_FPGA_230_classifier__best_mean_F1

***Script finished: 19:39:46

Time elapsed: 0:00:33.570722


### Train MORE

In [17]:
# if __name__ == "__main__":
    
#     print("Train More script\n")
#     logger.info("Train More\n")
    
#     model = train_loop(model, start_epoch=100, epochs_to_train=50)

# Test with DFire MINI Dataset: Train and Test

In [18]:
train_dfire_mini_loader = dataloaders.get_dfire_mini_train_loader()
test_dfire_mini_loader = dataloaders.get_dfire_mini_test_loader()


TRAIN DFire MINI dataset
DFire Removed wrong images: 0
DFire empty images: 20
DFire only smoke images: 45
DFire only fire images: 5
DFire smoke and fire images: 30

Test dataset len: 100

TEST DFire MINI dataset
DFire Removed wrong images: 0
DFire empty images: 6
DFire only smoke images: 13
DFire only fire images: 2
DFire smoke and fire images: 9

Test dataset len: 30


### Load Checkpoint with Best F1 Mean

In [19]:
model_path = config.WEIGHTS_FOLDER + config.MODEL + '_classifier__best_mean_F1.pt'

In [20]:
utils.load_checkpoint(
    model_path, 
    model=model, 
    optimizer=optimizer, 
    scheduler=scheduler, 
    device=config.DEVICE)

Loading Model. Trained during 2 epochs


2

### Whole Test Loader, to check it is the same as training

In [21]:
with torch.no_grad():
    val_losses, val_metrics = val_epoch.eval_fn(
        loader=val_loader, 
        model=model,                         
        loss_fn=loss_fn,
        device=config.DEVICE)

Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  3.12it/s]

Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
80.244      |41.026      |39.218      
SMOKE -> Precision: 0.4896 - Recall: 1.0000 - Accuracy: 0.4896 - F1: 0.6573
FIRE -> Precision: 0.0000 - Recall: 0.0000 - Accuracy: 0.6979 - F1: 0.0000


In [22]:
logger.info('\nTesting with FULL TEST LOADER')  
#logger.info(val_losses)
logger.info(val_metrics)

### Train DFire MINI

In [23]:
with torch.no_grad():
    val_losses, val_metrics = val_epoch.eval_fn(
        loader=train_dfire_mini_loader, 
        model=model,                         
        loss_fn=loss_fn,
        device=config.DEVICE)

Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 81.55it/s]

Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
1.217       |0.562       |0.655       
SMOKE -> Precision: 0.7500 - Recall: 1.0000 - Accuracy: 0.7500 - F1: 0.8571
FIRE -> Precision: 0.0000 - Recall: 0.0000 - Accuracy: 0.6500 - F1: 0.0000


In [24]:
logger.info('\nTesting with DFire MINI TRAIN after LOADING F1 Best Mean CHECKPOINT')  
#logger.info(val_losses)
logger.info(val_metrics)

### Test DFire MINI

In [25]:
with torch.no_grad():
    val_losses, val_metrics = val_epoch.eval_fn(
        loader=test_dfire_mini_loader, 
        model=model,                         
        loss_fn=loss_fn,
        device=config.DEVICE)

Validating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 70.85it/s]

Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
1.230       |0.563       |0.667       
SMOKE -> Precision: 0.7333 - Recall: 1.0000 - Accuracy: 0.7333 - F1: 0.8462
FIRE -> Precision: 0.0000 - Recall: 0.0000 - Accuracy: 0.6333 - F1: 0.0000


In [26]:
logger.info('\nTesting with DFire MINI TEST after LOADING F1 Best Mean CHECKPOINT')  
#logger.info(val_losses)
logger.info(val_metrics)

# Convert the Model to BIPOLAR OUT

In [27]:
import brevitas.nn as qnn
import torch.nn as nn

In [28]:
class CNV_BIPOLAR_OUT(nn.Module):
    def __init__(self, base_model):
        super(CNV_BIPOLAR_OUT, self).__init__()
        self.base_model = base_model
        self.qnt_output = qnn.QuantIdentity(
            quant_type='binary', 
            scaling_impl_type='const',
            bit_width=1, min_val=-1.0, max_val=1.0)

    def forward(self, x):
        x = self.base_model(x)
        x = self.qnt_output(x)
        return x

In [29]:
cnv_bipolar_out = CNV_BIPOLAR_OUT(model).to(config.DEVICE)

### New Evaluation for BIPOLAR Out Model

In [30]:
from tqdm import tqdm

def eval_bipolar_fn(loader, model, device):
    
    model.eval()
    loop = tqdm(loader, desc='Validating', leave=True)

    for batch_idx, (x, y) in enumerate(loop):
        x, y = x.to(device), y.to(device)
        yhat = model(x)

        # print(y.shape)
        # print(yhat.shape)
        
        yhat[yhat < 1] = 0
    
        metrics.precision_metric.update(yhat, y)
        metrics.recall_metric.update(yhat, y)
        metrics.accuracy_metric.update(yhat, y)
        metrics.f1_metric.update(yhat, y)
   
    precision = metrics.precision_metric.compute()
    recall = metrics.recall_metric.compute()
    accuracy = metrics.accuracy_metric.compute()
    f1 = metrics.f1_metric.compute()
    
    metrics.precision_metric.reset()
    metrics.recall_metric.reset()
    metrics.accuracy_metric.reset()
    metrics.f1_metric.reset()

    print(f'SMOKE -> Precision: {precision[0]:.4f} - Recall: {recall[0]:.4f} - Accuracy: {accuracy[0]:.4f} - F1: {f1[0]:.4f}')
    print(f'FIRE -> Precision: {precision[1]:.4f} - Recall: {recall[1]:.4f} - Accuracy: {accuracy[1]:.4f} - F1: {f1[1]:.4f}')
    
    return (
        {
        'Accuracy': [accuracy[0].item(), accuracy[1].item()],
        'Precision': [precision[0].item(), precision[1].item()],
        'Recall': [recall[0].item(), recall[1].item()],
        'F1': [f1[0].item(), f1[1].item()] 
        }
    )

### Full DS

In [31]:
cnv_bipolar_out.eval()
with torch.no_grad():
    val_metrics = eval_bipolar_fn(
        loader=val_loader, 
        model=cnv_bipolar_out,                         
        device=config.DEVICE)

Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  3.11it/s]

SMOKE -> Precision: 0.4896 - Recall: 1.0000 - Accuracy: 0.4896 - F1: 0.6573
FIRE -> Precision: 0.0000 - Recall: 0.0000 - Accuracy: 0.6979 - F1: 0.0000


### Mini Train

In [32]:
with torch.no_grad():
    val_metrics = eval_bipolar_fn(
        loader=train_dfire_mini_loader, 
        model=cnv_bipolar_out,                         
        device=config.DEVICE)

Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 82.83it/s]

SMOKE -> Precision: 0.7500 - Recall: 1.0000 - Accuracy: 0.7500 - F1: 0.8571
FIRE -> Precision: 0.0000 - Recall: 0.0000 - Accuracy: 0.6500 - F1: 0.0000


### Mini Test

In [33]:
with torch.no_grad():
    val_metrics = eval_bipolar_fn(
        loader=test_dfire_mini_loader, 
        model=cnv_bipolar_out,                         
        device=config.DEVICE)

Validating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 72.67it/s]

SMOKE -> Precision: 0.7333 - Recall: 1.0000 - Accuracy: 0.7333 - F1: 0.8462
FIRE -> Precision: 0.0000 - Recall: 0.0000 - Accuracy: 0.6333 - F1: 0.0000


# Export Bipolar to QONNX

In [34]:
save_f1_name = 'best_mean_F1'
save_bipolar_onnx = config.ONNX_FOLDER + config.MODEL + '_classifier__' + save_f1_name + '__BIPOLAR_Out'
utils.export_onnx(cnv_bipolar_out, (1, config.NUM_CHANNELS, config.IMG_H, config.IMG_W), save_bipolar_onnx, config.DEVICE)

Model exported to ONNX: experiments_bed_evolution/70_brevitas__230_manual_old_SmallBig__128_ds/onnx/BED_FPGA_230_classifier__best_mean_F1__BIPOLAR_Out
